### instalar dependencias

In [ ]:
!pip install smart-open
!pip install --upgrade gensim   
!pip install nltk
!pip install pyLDAvis
!pip install pandas

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
#importar librerías
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
# settings en nltk: tokenizador y stopwords
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('spanish'))

In [ ]:
import pickle 
import random

In [ ]:
# Mostrar la ruta del archivo indiferente del sistema operativo
import os
os.getcwd()

In [ ]:
main_path="C:\\Users\\erojas\\Desktop\\Downloads\\"
data_path="\\data\\03 - Trusted"

In [ ]:
os.chdir(main_path + data_path)

In [ ]:
data2=pickle.load( open(r'Fuente_2_vista_minable.pickle', "rb" ) )
data2.head()

In [ ]:
# Renombrar las columnas 
data2  = data2.rename(columns={
     'year':                        "periodo", 
     
     
})

In [ ]:
print(list(data2.columns))

In [ ]:
data2.shape

### Tokenización

In [ ]:
# función general para preparación de datos: tokenización, remoción caracteres especiales
# minusculas.... no incluido stemming y lematización.
def textprep(line):
    tokens = nltk.word_tokenize(str(line))
    tokens = [w.lower() for w in tokens if len(w)>1]
    tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in tokens]
    tokens = [w for w in tokens if w not in stop_words_nltk] 
    return tokens

In [ ]:
from nltk.tokenize import sent_tokenize

text=str(data2["mision_vision"])
tokenized_sent= sent_tokenize(text)
print(tokenized_sent)

In [ ]:
tokens = [t for t in text.split()]

freq = nltk.FreqDist(tokens)


for key,val in freq.items():

    print (str(key) + ':' + str(val))

In [ ]:
freq.plot(20, cumulative=False)

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
stop_words = stopwords.words('spanish')

clean_tokens = tokens[:]

sr = stopwords.words('spanish')

for token in tokens:

    if token in stopwords.words('spanish'):

        clean_tokens.remove(token)

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
stop_words = stopwords.words('spanish')

clean_tokens = tokens[:]

sr = stopwords.words('spanish')

for token in tokens:

    if token in ["La", "a", "o", "la", "del", "de", "es", "ser", "una", "infraestructura","nacional", "como", "mision","vision" ]  :

        clean_tokens.remove(token)

In [ ]:
freq2 = nltk.FreqDist(clean_tokens)


for key,val in freq.items():

    print (str(key) + ':' + str(val))

In [ ]:
freq2.plot(20,cumulative=False)

#### Stemming

In [ ]:
#stemming

words  = nltk.tokenize.WhitespaceTokenizer().tokenize(str(data2))
df = pd.DataFrame()
df['OriginalWords'] = pd.Series(words)
#porter's stemmer
porterStemmedWords = [nltk.stem.PorterStemmer().stem(word) for word in words]
df['PorterStemmedWords'] = pd.Series(porterStemmedWords)
#SnowBall stemmer
snowballStemmedWords = [nltk.stem.SnowballStemmer("spanish").stem(word) for word in words]
df['SnowballStemmedWords'] = pd.Series(snowballStemmedWords)
df

#### Lematización

In [ ]:
nltk.download('wordnet')

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lem= WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer
stem = PorterStemmer()

word = "misiones"
print("lemmatized word:", lem.lemmatize(word,"v"))
print("Stemmed Word:",stem.stem(word))

In [ ]:
words  = nltk.tokenize.WhitespaceTokenizer().tokenize(str(data2))
df = pd.DataFrame()
df['OriginalWords'] = pd.Series(words)
#WordNet Lemmatization
wordNetLemmatizedWords = [nltk.stem.WordNetLemmatizer().lemmatize(word) for word in words]
df['WordNetLemmatizer'] = pd.Series(wordNetLemmatizedWords)
df

### LDA

In [ ]:
# función general para preparación de datos: tokenización, remoción caracteres especiales
# minusculas.... no incluido stemming y lematización.
def textprep(line):
    tokens = nltk.word_tokenize(str(line))
    tokens = [w.lower() for w in tokens if len(w)>1]
    tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in tokens]
    tokens = [w for w in tokens if w not in stop_words_nltk] 
    return tokens

#### Producto (nombre_producto)

In [ ]:
# creación de columna con tokenización de una columna de interés especifica
data2['tokens_nombre_producto'] = data2.apply(lambda row: textprep(row['nombre_producto']), axis=1)
data2.head()

In [ ]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
dictionary = Dictionary(data2.tokens_nombre_producto)

In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in data2.tokens_nombre_producto]

In [ ]:
print (corpus) #no correr

In [ ]:
print(dictionary)

In [ ]:
# libreria para paralelizar
import multiprocessing as mp
import time

t0 = time.time()
pool = mp.Pool(mp.cpu_count())
doc_term_matrix = pool.map(dictionary.doc2bow, [sentence for sentence in data2.tokens_nombre_producto])
pool.close()
print(time.time()-t0)

In [ ]:
doc_term_matrix

In [ ]:
from gensim.models.ldamulticore import LdaMulticore

t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=20, id2word = dictionary, passes=10, workers=10)
print(time.time()-t0)

In [ ]:
def assigntopic(doc):
    vector = lda_model[dictionary.doc2bow(doc)] 
    # opción 1: todos los tópicos ordenados de mayor a menor, podria ser topN tambien asi: return vector[:5] n=5
    vector = sorted(vector, key=lambda item: -item[1])
    # opción 2: asignar el tópico mayor a cada documento
    #vector = max(vector,key=lambda item: item[1])
    return vector

In [ ]:
data2['topics'] = data2.apply(lambda row: assigntopic(row['tokens_nombre_producto']), axis=1)
data2.head()

In [ ]:
# Mostrar los términos y sus pesos de un documento
print(list(lda_model[doc_term_matrix[0]]))

In [ ]:
print (lda_model)

In [ ]:
# Mostrar los términos más relevantes de los tópicos más relevantes tópico y sus pesos
print(lda_model.print_topics(num_topics=10, num_words=5))

In [ ]:
lda_topic_assignment = [max(p,key=lambda item: item[1]) for p in lda_model[corpus]]

In [ ]:
import pyLDAvis.gensim_models as gensim
import pyLDAvis

t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensim.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
print(time.time()-t0)
vis

In [ ]:
pyLDAvis.save_html(vis, 'lda_visualizatioproducto.html')

###### modelo HDP con Producto

In [ ]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
common_dictionary = Dictionary(data2.tokens_nombre_producto)

In [ ]:
# Bag-of-words representation of the documents.
common_corpus = [dictionary.doc2bow(doc) for doc in data2.tokens_nombre_producto]

In [ ]:
from gensim.models import HdpModel

hdp = HdpModel(common_corpus, common_dictionary)

In [ ]:
vis_data = gensim.prepare(hdp, common_corpus, common_dictionary)
pyLDAvis.display(vis_data)

In [ ]:
pyLDAvis.save_html(vis_data, 'hdp_visualizationproducto.html')

#### Pesrpectiva estrategica

In [ ]:
# creación de columna con tokenización de una columna de interés especifica
data2['tokens_pers_estrategica'] = data2.apply(lambda row: textprep(row['pers_estrategica']), axis=1)
data2.head()

### Construir el BoW (diccionario) de términos

In [ ]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
dictionary = Dictionary(data2.tokens_pers_estrategica)

In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in data2.tokens_pers_estrategica]

##### no correr archivo pesado

In [ ]:
print(corpus)

In [ ]:
print(dictionary)


### Construir matriz de documentos vs términos

In [ ]:
# libreria para paralelizar
import multiprocessing as mp
import time

t0 = time.time()
pool = mp.Pool(mp.cpu_count())
doc_term_matrix = pool.map(dictionary.doc2bow, [sentence for sentence in data2.tokens_pers_estrategica])
pool.close()
print(time.time()-t0)

In [ ]:
doc_term_matrix

### Construir modelo LDA

In [ ]:
from gensim.models.ldamulticore import LdaMulticore

t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=20, id2word = dictionary, passes=10, workers=10)
print(time.time()-t0)

In [ ]:
def assigntopic(doc):
    vector = lda_model[dictionary.doc2bow(doc)] 
    # opción 1: todos los tópicos ordenados de mayor a menor, podria ser topN tambien asi: return vector[:5] n=5
    vector = sorted(vector, key=lambda item: -item[1])
    # opción 2: asignar el tópico mayor a cada documento
    #vector = max(vector,key=lambda item: item[1])
    return vector

In [ ]:
data2['topics2'] = data2.apply(lambda row: assigntopic(row['tokens_pers_estrategica']), axis=1)
data2.head()

Ejemplos de tópicos del modelo

In [ ]:
# Mostrar los términos y sus pesos de un documento
print(list(lda_model[doc_term_matrix[0]]))



In [ ]:
print (lda_model)

In [ ]:
# Mostrar los términos más relevantes de los tópicos más relevantes tópico y sus pesos
print(lda_model.print_topics(num_topics=10, num_words=5))

In [ ]:
lda_topic_assignment = [max(p,key=lambda item: item[1]) for p in lda_model[corpus]]

### Visualización de todos los tópicos

In [ ]:
import pyLDAvis.gensim_models as gensim
import pyLDAvis

t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensim.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
print(time.time()-t0)
vis

### Guardar la visualización en un archivo HTML 

In [ ]:
pyLDAvis.save_html(vis, 'lda_visualizationestrategica2.0.html')

###### HDP con Perpectiva Estrategica

In [ ]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
dictionary = Dictionary(data2.tokens_pers_estrategica)

In [ ]:
# Bag-of-words representation of the documents.
common_corpus = [dictionary.doc2bow(doc) for doc in data2.tokens_pers_estrategica]

In [74]:
from gensim.models import HdpModel

hdp = HdpModel(common_corpus, common_dictionary)

KeyboardInterrupt: 

In [ ]:
vis_data2 = gensim.prepare(hdp, common_corpus, common_dictionary)
pyLDAvis.display(vis_data2)

In [ ]:
pyLDAvis.save_html(vis_data2, 'hdp_visualizationestrategica.html')

### Misión_visión

In [ ]:
# creación de columna con tokenización de una columna de interés especifica # 3 minutos y 30 segundos Cargando
data2['tokens_mision_vision'] = data2.apply(lambda row: textprep(row['mision_vision']), axis=1)
data2.head()

In [ ]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
dictionary = Dictionary(data2.tokens_mision_vision)

In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in data2.tokens_mision_vision]

In [ ]:
# libreria para paralelizar
import multiprocessing as mp
import time

t0 = time.time()
pool = mp.Pool(mp.cpu_count())
doc_term_matrix = pool.map(dictionary.doc2bow, [sentence for sentence in data2.tokens_mision_vision])
pool.close()
print(time.time()-t0)

In [ ]:
from gensim.models.ldamulticore import LdaMulticore 
#3 minutos cargando
t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=20, id2word = dictionary, passes=10, workers=10)
print(time.time()-t0)

In [ ]:
def assigntopic(doc):
    vector = lda_model[dictionary.doc2bow(doc)] 
    # opción 1: todos los tópicos ordenados de mayor a menor, podria ser topN tambien asi: return vector[:5] n=5
    vector = sorted(vector, key=lambda item: -item[1])
    # opción 2: asignar el tópico mayor a cada documento
    #vector = max(vector,key=lambda item: item[1])
    return vector

In [ ]:
data2['topics3'] = data2.apply(lambda row: assigntopic(row['tokens_mision_vision']), axis=1)
data2.head()
#1 minuto y medio cargado

In [ ]:
# Mostrar los términos y sus pesos de un documento
print(list(lda_model[doc_term_matrix[0]]))


In [ ]:
print (lda_model)

In [ ]:
# Mostrar los términos más relevantes de los tópicos más relevantes tópico y sus pesos
print(lda_model.print_topics(num_topics=10, num_words=5))

In [ ]:
lda_topic_assignment = [max(p,key=lambda item: item[1]) for p in lda_model[corpus]]

In [ ]:
import pyLDAvis.gensim_models as gensim
import pyLDAvis

t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensim.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
print(time.time()-t0)
vis

In [ ]:
pyLDAvis.save_html(vis, 'lda_visualizationmisiovision2.0.html')

## Fitting the HDP model

In [ ]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
common_dictionary = Dictionary(data2.tokens_mision_vision)

In [ ]:
# Bag-of-words representation of the documents.
common_corpus = [dictionary.doc2bow(doc) for doc in data2.tokens_mision_vision]

In [ ]:
from gensim.models import HdpModel

hdp = HdpModel(common_corpus, common_dictionary)

In [ ]:
vis_data3 = gensim.prepare(hdp, common_corpus, common_dictionary)
pyLDAvis.display(vis_data)

In [ ]:
pyLDAvis.save_html(vis_data3, 'hdp_visualizationmisiovision.html')